# Agent Comparison

This notebook compares different AI agent implementations, analyzing their performance characteristics and decision-making patterns.

## Setup

In [ ]:
import sys
import time
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.players import (
    RandomPlayer,
    GreedyPlayer,
    HeuristicPlayer,
    MCTSPlayer,
)
from src.players.mcts_player import MCTSConfig
from src.simulation import GameRunner, MatchConfig, MatchRunner
from src.analysis import BalanceAnalyzer, BalanceConfig, create_matchup_matrix

print("Imports successful!")

## Agent Overview

| Agent | Strategy | Complexity | Speed |
|-------|----------|------------|-------|
| Random | Random actions | O(1) | Very Fast |
| Greedy | Best immediate value | O(n) | Fast |
| Heuristic | Synergy + Evolution tracking | O(n) | Fast |
| MCTS | Monte Carlo Tree Search | O(simulations * depth) | Slow |

## Single Game Demonstration

Run a single game to see the agents in action.

In [ ]:
# Create players
players = [
    HeuristicPlayer(player_id=0, name="Heuristic"),
    GreedyPlayer(player_id=1, name="Greedy"),
]

# Run a single game
runner = GameRunner(players=players, seed=42, log_events=True, max_turns=30)
result = runner.run_game()

print(f"Game completed in {result.turns} turns")
if result.winner_id is not None:
    print(f"Winner: {players[result.winner_id].name} ({result.winner_type})")
else:
    print("Result: Draw")

print(f"\nPlayer Statistics:")
for pid, stats in result.player_stats.items():
    print(f"  {players[pid].name}:")
    print(f"    Cards bought: {stats.cards_bought}")
    print(f"    Cards played: {stats.cards_played}")
    print(f"    Damage dealt: {stats.damage_dealt}")

## Speed Comparison

Compare decision-making speed of different agents.

In [ ]:
def benchmark_agent(factory, name, num_games=10):
    """Benchmark an agent's game speed."""
    opponent_factory = lambda pid: RandomPlayer(player_id=pid, seed=pid)

    config = MatchConfig(
        num_games=num_games,
        player_factories=[factory, opponent_factory],
        base_seed=42,
    )

    runner = MatchRunner()

    start = time.time()
    result = runner.run_match(config)
    elapsed = time.time() - start

    return {
        "name": name,
        "games": num_games,
        "total_time": elapsed,
        "time_per_game": elapsed / num_games,
        "win_rate": result.win_rates.get(name.lower(), 0),
    }


# Benchmark each agent type
agents_to_test = [
    (lambda pid: RandomPlayer(player_id=pid, seed=pid), "random"),
    (lambda pid: GreedyPlayer(player_id=pid), "greedy"),
    (lambda pid: HeuristicPlayer(player_id=pid), "heuristic"),
]

print("Benchmarking agents (10 games each vs Random)...\n")
results = []
for factory, name in agents_to_test:
    bench = benchmark_agent(factory, name)
    results.append(bench)
    print(f"{name}:")
    print(f"  Total time: {bench['total_time']:.2f}s")
    print(f"  Per game: {bench['time_per_game'] * 1000:.1f}ms")
    print(f"  Win rate vs Random: {bench['win_rate']:.1%}")
    print()

## MCTS Configuration Tuning

Explore how MCTS parameters affect performance.

In [ ]:
# Test different simulation counts
simulation_counts = [10, 25, 50]
mcts_results = []

print("Testing MCTS with different simulation counts...\n")

for num_sims in simulation_counts:
    config = MCTSConfig(num_simulations=num_sims, max_rollout_depth=5)
    factory = lambda pid, c=config: MCTSPlayer(player_id=pid, config=c, seed=pid)

    bench = benchmark_agent(factory, "mcts", num_games=5)
    bench["simulations"] = num_sims
    mcts_results.append(bench)

    print(f"MCTS (sims={num_sims}):")
    print(f"  Per game: {bench['time_per_game'] * 1000:.1f}ms")
    print(f"  Win rate vs Random: {bench['win_rate']:.1%}")
    print()

## Head-to-Head Tournament

Run a full round-robin tournament between all agents.

In [ ]:
# Create factories for all agents
all_factories = [
    lambda pid: RandomPlayer(player_id=pid, seed=pid * 100),
    lambda pid: GreedyPlayer(player_id=pid),
    lambda pid: HeuristicPlayer(player_id=pid),
]

# Run round-robin
print("Running round-robin tournament (20 games per matchup)...\n")

config = BalanceConfig(games_per_matchup=20, base_seed=42)
analyzer = BalanceAnalyzer(config)
report = analyzer.run_analysis(all_factories, verbose=True)

print("\n" + report.summary())

## Matchup Matrix with Statistical Analysis

In [ ]:
# Run tournament and get raw results
runner = MatchRunner()
raw_results = runner.run_round_robin(
    player_factories=all_factories,
    games_per_matchup=30,
    base_seed=123,
)

# Create matchup matrix with statistical analysis
matrix = create_matchup_matrix(raw_results)

print(matrix.summary())
print("\n" + "=" * 60)
print("\nRankings:")
for rank, (ptype, rate) in enumerate(matrix.get_rankings(), 1):
    print(f"  {rank}. {ptype}: {rate:.1%} average win rate")

## Agent Behavior Analysis

Analyze decision patterns of different agents.

In [ ]:
# Run games and collect detailed statistics
def analyze_agent_behavior(factory, name, num_games=10):
    """Analyze an agent's behavior patterns."""
    opponent = lambda pid: RandomPlayer(player_id=pid, seed=pid + 500)

    config = MatchConfig(
        num_games=num_games,
        player_factories=[factory, opponent],
        base_seed=999,
        log_events=False,
    )

    runner = MatchRunner()
    result = runner.run_match(config)

    # Aggregate stats from all games
    total_bought = 0
    total_played = 0
    total_evolutions = 0

    for game_result in result.results:
        for pid, stats in game_result.player_stats.items():
            if stats.player_type == name.lower():
                total_bought += stats.cards_bought
                total_played += stats.cards_played
                total_evolutions += stats.evolutions

    return {
        "name": name,
        "games": num_games,
        "avg_cards_bought": total_bought / num_games,
        "avg_cards_played": total_played / num_games,
        "avg_evolutions": total_evolutions / num_games,
        "win_rate": result.win_rates.get(name.lower(), 0),
    }


# Analyze each agent
print("Agent Behavior Analysis:\n")

for factory, name in agents_to_test:
    behavior = analyze_agent_behavior(factory, name, num_games=15)
    print(f"{name}:")
    print(f"  Avg cards bought per game: {behavior['avg_cards_bought']:.1f}")
    print(f"  Avg cards played per game: {behavior['avg_cards_played']:.1f}")
    print(f"  Avg evolutions per game: {behavior['avg_evolutions']:.2f}")
    print(f"  Win rate: {behavior['win_rate']:.1%}")
    print()

## Conclusions

### Agent Strengths and Weaknesses

| Agent | Strengths | Weaknesses |
|-------|-----------|------------|
| Random | Fast, unpredictable | No strategy |
| Greedy | Good immediate value | No long-term planning |
| Heuristic | Synergy building, evolution tracking | Fixed strategy |
| MCTS | Adaptive, looks ahead | Slow, resource intensive |

### Recommendations

1. **For development testing**: Use Random or Greedy for fast iteration
2. **For balance testing**: Use Heuristic as the baseline
3. **For strong opponents**: Tune MCTS with appropriate simulation count
4. **For analysis**: Compare agents across multiple seeds for statistical validity